In [57]:
import rioxarray as rioxr
from dataclasses import asdict
from dscreator.cfarray.attributes import DatasetAttrsGrid, VariableAttrs, LongitudeAttrs, LatitudeAttrs
from dscreator import utils
import xarray as xr
import os
from dscreator.storage import get_storage_handler
from dscreator.config import SETTINGS

urchin_da = rioxr.open_rasterio(
    "S_droebachiensis_final_biomass_sqm_wgs84.tif",
    masked=False
).drop_vars("band")

name = "predicted_density"
urchin_da = urchin_da.squeeze("band", drop=True)
urchin_da.name = name
urchin_da.attrs = asdict(VariableAttrs(long_name="Urchin Predicted Density", units="", short_name=name))


In [58]:
urchin_da.x.attrs = asdict(LongitudeAttrs())
urchin_da.y.attrs = asdict(LatitudeAttrs())
urchin_da = urchin_da.drop_vars("spatial_ref")

In [59]:
ds = xr.merge([urchin_da])
ds.attrs = asdict(
    DatasetAttrsGrid(
        title="S_droebachiensis_final_biomass_sqm_wgs84",
        title_no="S_droebachiensis_final_biomass_sqm_wgs84",
        summary="S_droebachiensis_final_biomass_sqm_wgs84",
        summary_no="S_droebachiensis_final_biomass_sqm_wgs84",
        # https://gcmd.earthdata.nasa.gov/KeywordViewer
        keywords=",".join(
            [
                #"GCMDSK:EARTH SCIENCE > BIOLOGICAL CLASSIFICATION > SPECIES",
                #"GCMDSK:EARTH SCIENCE > LAND SURFACE > VEGETATION > PLANT FUNCTIONAL TYPES",
                #"GCMDSK:EARTH SCIENCE > LAND SURFACE > VEGETATION > VEGETATION INDEX",
            ]
        ),
        keywords_vocabulary=",".join(
            [
                #"GCMDSK:GCMD Science Keywords:https://gcmd.earthdata.nasa.gov/kms/concepts/concept_scheme/sciencekeywords",
                #"GCMDLOC:GCMD Locations:https://gcmd.earthdata.nasa.gov/kms/concepts/concept_scheme/locations",
            ]
        ),
        time_coverage_end="",
        time_coverage_start="",
        iso_topic_category="environment",
        spatial_representation="grid",
        date_created=utils.iso_now(),
        project="Refarktis",
        geospatial_lon_min=float(ds.x.min()),
        geospatial_lon_max=float(ds.x.max()),
        geospatial_lat_min=float(ds.y.min()),
        geospatial_lat_max=float(ds.y.max()),
        history=f"",
    )
)
ds.attrs["id"] = "522d5685-7157-407c-bb22-6a882ed5cbd8"

In [61]:
SETTINGS.storage_path = os.path.join(os.getcwd(), "..", "catalog")
#SETTINGS.storage_path = "gs://nivatest-1-senda"

ds['x'] = ds['x'].astype('float32')
ds['y'] = ds['y'].astype('float32')
# For transfer directly into the test thredds server use

get_storage_handler(
    grouping="refarktis",
    dataset_name="S_droebachiensis_final_biomass_sqm_wgs84",
    filename_prefix="density").save_dataset(ds)

KeyboardInterrupt: 